![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/annotation/text/english/language-translation/Multilingual_Translation_with_M2M100.ipynb)

# Multilingual Translation with M2M100

## Colab Setup

In [1]:
!wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.0
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.8/564.8 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 23.8 MB/s eta 0:00:00


In [2]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 5.3.0
Apache Spark version: 3.2.3


# Define Spark NLP pipeline

**A sample text in Chinese - we'll translate it to English**

In [3]:
text = """除了是北方之王之外，约翰·斯诺还是一位英国医生，也是麻醉和医疗卫生发展的领导者。 他被认为是第一个利用数据治愈 1854 年霍乱爆发的人。"""

In [4]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

m2m100 = M2M100Transformer.pretrained() \
    .setInputCols(["document"]) \
    .setMaxOutputLength(50) \
    .setOutputCol("generation") \
    .setSrcLang("zh") \
    .setTgtLang("en")

tl_pipeline = Pipeline(
    stages=[documentAssembler, m2m100]
    )

m2m100_418M download started this may take some time.
Approximate size to download 2.8 GB
[OK!]


# Light Pipeline version

Let's create the light Pipiline

In [7]:
empty_df = spark.createDataFrame([[""]]).toDF('text')
pipeline_model = tl_pipeline.fit(empty_df)
model = LightPipeline(pipeline_model)
res = model.fullAnnotate(text)

visualize the results

In [16]:
print ('Original:', text, '\n\n')

print ('Translated:\n')
for sentence in res[0]['generation']:
  print (sentence.result)

Original: 除了是北方之王之外，约翰·斯诺还是一位英国医生，也是麻醉和医疗卫生发展的领导者。 他被认为是第一个利用数据治愈 1854 年霍乱爆发的人。 


Translated:

In addition to being the King of the North, John Snow was also a British doctor and a leader in the development of anesthesia and health care. he was considered the first person to use data to cure the 1854 cholera outbreak.
